In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, Trainer
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
from datasets import load_dataset
dataset = load_dataset(
    "parquet",
    data_files={"train": "4701_cleaned.parquet"},
    split="train"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
sample = dataset.select(range(min(5, len(dataset))))
example_convo = sample['conversations'][0]
print(f"Type of conversation: {type(example_convo)}")
print(f"Example conversation structure: {example_convo}")

Type of conversation: <class 'str'>
Example conversation structure: [{'from': 'human', 'value': 'SYSTEM: Respond with hello world and your name\nSYSTEM: Respond with hello world and your name'}
 {'from': 'gpt', 'value': "Hello world! I'm Lucy, a Minecraft bot. !stats"}
 {'from': 'human', 'value': 'SYSTEM: STATS\n- Position: x: -25.50, y: 66.00, z: -75.50\n\n- Health: 20 / 20\n- Hunger: 20 / 20\n- Current Action: Idle\n- Nearby Human Players: MrBigFinger\nAgent Modes:\n- self_preservation(ON)\n- unstuck(ON)\n- cowardice(OFF)\n- self_defense(ON)\n- hunting(ON)\n- item_collecting(ON)\n- torch_placing(ON)\n- elbow_room(ON)\n- idle_staring(ON)\n- \nSYSTEM: Respond with hello world and your name'}
 {'from': 'gpt', 'value': "Hey there! I'm Lucy, hanging out in the plains. !entities"}]


In [7]:
dataset[5]["conversations"]

'[{\'from\': \'human\', \'value\': \'SYSTEM: Respond with hello world and your name\'}\n {\'from\': \'gpt\', \'value\': "Hey there! I\'m Lucy, a Minecraft bot. !stats"}\n {\'from\': \'human\', \'value\': \'SYSTEM: STATS\\n- Position: x: -25.50, y: 66.00, z: -75.50\\n\\n- Health: 20 / 20\\n- Hunger: 20 / 20\\n- Current Action: Idle\\n- Nearby Human Players: MrBigFinger\\nAgent Modes:\\n- self_preservation(ON)\\n- unstuck(ON)\\n- cowardice(OFF)\\n- self_defense(ON)\\n- hunting(ON)\\n- item_collecting(ON)\\n- torch_placing(ON)\\n- elbow_room(ON)\\n- idle_staring(ON)\\n- \\nMrBigFinger: So Lucy, can you fetch me some wood to start off our adventure in Minecraft?\'}\n {\'from\': \'gpt\', \'value\': "Sure thing! Let me check what\'s around me first. !nearbyBlocks"}\n {\'from\': \'human\', \'value\': \'SYSTEM: NEARBY_BLOCKS\\n- grass_block\\n- short_grass\\n- dirt\\n- stone\\n- copper_ore\\n- granite\\n- water\\n- coal_ore\\n- lapis_ore\\n- dandelion\\n- poppy\\n- diorite\\n- birch_leaves\\n-

In [9]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token=True,
)

# Chain-of-Thought augmentation
def format_chain_of_thought(example):
    new_convo = []
    for message in example["conversations"]:
        if message[0] == "gpt":
            reasoning = f"Let's think step-by-step to solve the instruction: '{example['conversations'][0]['value']}'"
            new_convo.append({"from": "gpt", "value": reasoning})
        new_convo.append(message)
    return {"conversations": new_convo}

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
    )

# Load and process dataset
dataset = dataset.map(format_chain_of_thought)
dataset = dataset.map(formatting_prompts_func, batched=True)
dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.


Map:   0%|          | 0/1879 [00:00<?, ? examples/s]

Map:   0%|          | 0/11647 [00:00<?, ? examples/s]

Map:   0%|          | 0/11647 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 1,
        warmup_steps = 30,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        gradient_checkpointing = True,
    ),
)

In [9]:
import torch
torch.cuda.empty_cache()

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.441 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 11,647 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eh588 (eh588-cornell-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.978600
2,1.941100
3,1.857300
4,1.857500
5,1.858000
6,1.623200
7,1.529200
8,1.648400
9,1.792400
10,1.733600


Unsloth: Will smartly offload gradients to save VRAM!


In [13]:
model.save_pretrained("outputs/lucy/model")
tokenizer.save_pretrained("outputs/lucy/tokenizer")

('outputs/lucy/tokenizer/tokenizer_config.json',
 'outputs/lucy/tokenizer/special_tokens_map.json',
 'outputs/lucy/tokenizer/tokenizer.json')